In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with WTF model

In [2]:
from pathlib import Path

import src.io as io
import src.evaluation as evaluation
import src.evaluation.splits
import src.evaluation.cars

from src.algorithm.cars.wtf import WTF

## Datasets

In [3]:
## Uncomment corresponding directory.
## RETARGET True for Frappe, False for the others

# DATA_DIR, RETARGET = Path('../../../data/CARS/Mobile_Frappe/'), True
DATA_DIR, RETARGET = Path('../../../data/CARS/Food_com/'), False
# DATA_DIR, RETARGET = Path('../../../data/CARS/TripAdvisor/'), False

In [4]:
# Shouldn't need to change this info
INTERACTIONS = DATA_DIR / 'interactions.csv'

SEED = 123456
SEED2 = 78910

ITEM_ID = 'item'
USER_ID = 'user'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = WTF

K = [80]
MAX_IT = [10]
DEFAULT_CONTEXT_1 = [False]

# L2 = [100, 1000, 10000, 100000]
# V = [0, 0.5, 1]
# ALPHA = [100, 1000, 10000, 100000]

L2 = [1, 10, 50, 100, 200]
V = [0.75, 1]
ALPHA = [1, 10, 50, 100, 200]

HYPERPARAMS = {'k': K, 'l2': L2, 'v': V, 'alpha': ALPHA, 'max_iterations': MAX_IT, 'default_context_1': DEFAULT_CONTEXT_1}
HYPERPARAMS

{'k': [80],
 'l2': [1, 10, 50, 100, 200],
 'v': [0.75, 1],
 'alpha': [1, 10, 50, 100, 200],
 'max_iterations': [10],
 'default_context_1': [False]}

## Parse data

In [6]:
data = io.parse_interactions_with_context(INTERACTIONS, item_id=ITEM_ID, user_id=USER_ID)
data.df

,userId,itemId,daytime,weather,weekday
0,0,0,1,1,1
1,1,1,2,2,2
2,2,2,3,2,3
3,3,3,4,0,4
4,4,4,5,1,4
...,...,...,...,...,...
95997,109,0,3,1,1
95998,37,16,4,2,1
95999,180,33,2,2,1
96000,445,751,3,1,1


## Make train/val split for hyperparam tuning

In [7]:
train, test = evaluation.splits.context_leave_one_out_split(data, seed=SEED)

## Perform gridsearch on validation set

In [8]:
%%time
score, best_hyperparams = evaluation.cars.gridsearch(ALG, train, test, HYPERPARAMS, retarget=RETARGET)
f"Best score of {score} achieved with {best_hyperparams}."

  0%|          | 0/50 [00:00<?, ?it/s]

Training model WTF with hyperparameters {'k': 80, 'l2': 1, 'v': 0.75, 'alpha': 1, 'max_iterations': 10, 'default_context_1': False}


  0%|          | 0/10 [00:00<?, ?it/s]

Evaluating with 816 users
MRR@5 0.252
MRR@20 0.271
Average Recall@5 0.395
Average Recall@20 0.576
Training model WTF with hyperparameters {'k': 80, 'l2': 10, 'v': 0.75, 'alpha': 1, 'max_iterations': 10, 'default_context_1': False}


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Evaluate model with optimal hyperparams with KFolds Cross validation

In [9]:
%%time
alg = ALG(**best_hyperparams)
results = evaluation.cars.contextKFoldsEval(alg, data, nr_folds=5, seed=SEED2, retarget=RETARGET)

NameError: name 'best_hyperparams' is not defined